In [1]:
import os
import cv2
import glob
import random
import numpy as np
import imgaug as ia
import pandas as pd
import tensorflow as tf
from progressbar import * #进度条
from natsort import natsorted
import matplotlib.pyplot as plt
from imgaug import augmenters as iaa
from imgaug import parameters as iap
from skimage.transform import resize
from skimage.io import imread, imsave
from sklearn.metrics import confusion_matrix
from skimage import transform, filters, exposure

os.environ['CUDA_VISIBLE_DEVICES']='3'
%matplotlib inline

/home/lhw/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/lhw/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
label2int = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
int2lable = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [3]:
#由于数据集较大，采用tensorflow自带的TFRecord进行读取
def Trainset2TFRecord2(train_tfrecords_path, val_tfrecords_path, ratio=0.9, data_path='data'):
    files = natsorted(glob.glob(data_path + '/train/*'))
    total = len(files)
    train_num = int(total*ratio)
    val_num = total-train_num
    if (os.path.exists(train_tfrecords_path) and os.path.exists(val_tfrecords_path)):
        print('data have already processed')
        return train_num, val_num
    
    writer_train = tf.python_io.TFRecordWriter(train_tfrecords_path)
    writer_val = tf.python_io.TFRecordWriter(val_tfrecords_path)
    
    labels = pd.read_csv(data_path + '/trainLabels.csv')
    
    pbar = ProgressBar().start()
    for i, file_path in enumerate(files):
        pbar.update(int((i / (total - 1)) * 100))#进度条
        time.sleep(0.01)

        img_raw = tf.gfile.FastGFile(file_path, 'rb').read()
        label = label2int[labels.loc[i].label]

        example = tf.train.Example(features=tf.train.Features(feature={
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
        }))
        if i < train_num:
            writer_train.write(example.SerializeToString())
        else:
            writer_val.write(example.SerializeToString())
    writer_train.close()
    writer_val.close()
    pbar.finish()
    
    return train_num, val_num
def TFRecord2TrainData2(train_tfrecords_path, num_class, batch_size, num_epochs, shuffle=True):
    filename_queue = tf.train.string_input_producer([train_tfrecords_path], num_epochs=num_epochs)
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                   features={
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                       'label': tf.FixedLenFeature([], tf.int64)
                                   })
    img = tf.image.decode_png(features['img_raw'])  # 与方式一的不同点在于需要用decode_png/decode_jpeg解码
                                                    # output an RGB image. [height, width, channels]
    img = tf.reshape(img, [32, 32, 3])
    img = tf.image.resize_images(img, [224, 224])
    label = tf.cast(features['label'], tf.int32)
    label = tf.one_hot(label,num_class,1,0)
    if shuffle==True:
        img_batch, label_batch = tf.train.shuffle_batch([img, label], batch_size=batch_size,\
                                                        capacity=500 + 3 * batch_size, min_after_dequeue=500)
    else:
        img_batch, label_batch = tf.train.batch([img, label], batch_size=batch_size, capacity= 3 * batch_size)
    return img_batch, label_batch

In [4]:
train_tfrecords_path = 'data/train.tfrecord'
val_tfrecords_path = 'data/val.tfrecord'
train_num, val_num = Trainset2TFRecord2(train_tfrecords_path, val_tfrecords_path)
print(train_num, val_num)

data have already processed
45000 5000


In [23]:
#解码并查看效果
#img,label = TFRecord2TrainData(train_tfrecords_path, 10)
#img_batch, label_batch = tf.train.shuffle_batch([img, label], batch_size=500, capacity=50000, min_after_dequeue=1000, num_threads=1)
#capacity是队列的最大容量, min_after_dequeue是dequeue后最小的队列大小,这个代表队列中的元素大于它的时候就输出乱的顺序的batch
#num_threads是进行队列操作的线程数。
img_train_batch, label_train_batch = TFRecord2TrainData2(train_tfrecords_path, 10, 128, 5)
img_val_batch, label_val_batch = TFRecord2TrainData2(val_tfrecords_path, 10, 128, 5)#10类 batch_size=128, epoch = 5
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    #先产生！个看下效果
    '''
    for epoch in range(10):
        try:
            while not coord.should_stop():
                img_, label_ = sess.run([img_batch, label_batch]) 
                print(epoch)
        except tf.errors.OutOfRangeError:
            print("done")
        finally:
            coord.request_stop()
        coord.join(threads)
    ''' 
    #验证集上测试
    try:
        train_total_batch = int(train_num/128)
        val_total_bath = int(val_num/128)
        epoch = 0
        while not coord.should_stop():#使用 coord.should_stop()来查询是否应该终止所有线程
            #开始一个epoch的训练
            train_begin = 0
            val_begin = 0
            for i in range(train_total_batch):
                img_, label_ = sess.run([img_train_batch, label_train_batch]) 
                if train_begin == 0:
                    print("train")
                    print(epoch, img_.shape)
                    train_begin = 1
            for i in range(val_total_bath):
                img_, label_ = sess.run([img_val_batch, label_val_batch]) 
                if val_begin == 0:
                    print("val")
                    print(epoch, img_.shape)
                    val_begin = 1
            epoch+=1
    except tf.errors.OutOfRangeError:
        print("done")
    finally:       
        coord.request_stop()#使用coord.request_stop()来发出终止所有线程的命令
        coord.join(threads)#使用coord.join(threads)把线程加入主线程，等待threads结束。
    '''
    for i in range(10): #产生10个batch
        img_, label_ = sess.run([img_batch, label_batch])#每次输出queue中的一个batch
        plt.subplot(2, 5, i+1)
        label = int2lable[np.argmax(label_, axis=1)[0]]
        plt.title(label)
        plt.imshow(img_[0].astype('uint8'))
        i += 1
    plt.show()
    coord.request_stop()
    coord.join(threads)
    '''

train
0 (128, 224, 224, 3)
val
0 (128, 224, 224, 3)
train
1 (128, 224, 224, 3)
val
1 (128, 224, 224, 3)
train
2 (128, 224, 224, 3)
val
2 (128, 224, 224, 3)
train
3 (128, 224, 224, 3)
val
3 (128, 224, 224, 3)
train
4 (128, 224, 224, 3)
val
4 (128, 224, 224, 3)
train
5 (128, 224, 224, 3)
done


In [5]:
### 查看预训练模型中的参数 ###
pre_model_path = 'model/vgg19/vgg19.npy'
pre_model = np.load(pre_model_path, encoding = "bytes").item()
layers = list(pre_model.keys())#将dict转换为list  层的名称
#layer_1 = pre_model[layers[0]] #第一层中内容
#layer_1_w =  pre_model[layers[0]][0]  #第一层的权重
print(layers)
layer_1_0 =  pre_model[layers[0]][1] #第一层的偏差
print(pre_model[layers[0]][0].shape) #第一层权重的形状 (3, 3, 3, 64)
print(pre_model[layers[0]][1].shape) #第一层的偏差的形状 (64,)

['conv5_4', 'conv5_1', 'fc6', 'conv5_3', 'conv5_2', 'conv4_4', 'fc7', 'conv4_1', 'conv4_2', 'conv4_3', 'fc8', 'conv3_4', 'conv3_3', 'conv3_2', 'conv3_1', 'conv1_1', 'conv1_2', 'conv2_2', 'conv2_1']
(3, 3, 512, 512)
(512,)


In [6]:
class VGG_19:
    def __init__(self, x, classname, is_training, weights=None, sess=None):
        self.parameters = []
        self.vgg_mean = [103.939, 116.779, 123.68]
        self.build_model(x, classname, is_training)
        self.skip = ['fc7', 'fc8'] #需要重新训练的层
        self.probs = self.fc8
        if weights is not None and sess is not None:
            self.load_weights(weights, sess)
    
    def maxpool(self,name,input_data, trainable):
        out = tf.nn.max_pool(input_data,[1,2,2,1],[1,2,2,1],padding="SAME",name=name)
        return out
    
    def conv(self,name, input_data, out_channel, trainable):
        in_channel = input_data.get_shape()[-1]
        with tf.variable_scope(name):
            kernel = tf.get_variable("weights", [3, 3, in_channel, out_channel], dtype=tf.float32,trainable=False)
            biases = tf.get_variable("biases", [out_channel], dtype=tf.float32,trainable=False)
            conv_res = tf.nn.conv2d(input_data, kernel, [1, 1, 1, 1], padding="SAME")
            res = tf.nn.bias_add(conv_res, biases)
            #norm = tf.layers.batch_normalization(res, center=True, scale=True, training=trainable)
            out = tf.nn.relu(res, name=name)
        self.parameters += [kernel, biases]
        return out
    
    def fc(self,name,input_data,out_channel, relu_flag = True,trainable = True):
        shape = input_data.get_shape().as_list()
        if len(shape) == 4:
            size = shape[-1] * shape[-2] * shape[-3]
        else:size = shape[1]
        input_data_flat = tf.reshape(input_data,[-1,size])
        with tf.variable_scope(name):
            weights = tf.get_variable(name="weights",shape=[size,out_channel],dtype=tf.float32,trainable = trainable)
            biases = tf.get_variable(name="biases",shape=[out_channel],dtype=tf.float32,trainable = trainable)
            res = tf.matmul(input_data_flat,weights)
            if relu_flag == True:
                #norm = tf.layers.batch_normalization(res, center=True, scale=True, training=trainable)
                out = tf.nn.relu(tf.nn.bias_add(res,biases))
            else:
                out = res
        self.parameters += [weights, biases]
        return out
    
    def build_model(self, x, classnum, is_training):
        # Preprocess
        # Convert RGB to BGR opencv 读取的为BGR格式，所以不用转换
        #red, green, blue = tf.split(axis=3, num_or_size_splits=3, value=self.rgb)#RGB格式
        blue,  green, red = tf.split(axis=3, num_or_size_splits=3, value=x)#张量被切为三份
        self.bgr = tf.concat(axis=3, values=[blue - self.vgg_mean[0], green - self.vgg_mean[1],red - self.vgg_mean[2]])
        
        # Block 1
        #对于每个卷积层和全连接层中，不需要训练的权重全部被设置为trainable=False
        self.conv1_1 = self.conv("conv1_1",self.bgr,64,trainable=True)
        self.conv1_2 = self.conv("conv1_2",self.conv1_1,64,trainable=True)
        self.pool1 = self.maxpool("pool1",self.conv1_2,trainable=True)
        
        # Block 2
        self.conv2_1 = self.conv("conv2_1",self.pool1,128,trainable=True)
        self.conv2_2 = self.conv("conv2_2",self.conv2_1,128,trainable=True)
        self.pool2 = self.maxpool("pool2",self.conv2_2,trainable=True)  
        
        # Block 3
        self.conv3_1 = self.conv("conv3_1",self.pool2,256,trainable=True)
        self.conv3_2 = self.conv("conv3_2",self.conv3_1,256,trainable=True)
        self.conv3_3 = self.conv("conv3_3",self.conv3_2,256,trainable=True)
        self.conv3_4 = self.conv("conv3_4",self.conv3_3,256,trainable=True)
        self.pool3 = self.maxpool("pool3",self.conv3_4,trainable=True)  
        
        # Block 4
        self.conv4_1 = self.conv("conv4_1",self.pool3,512,trainable=True)
        self.conv4_2 = self.conv("conv4_2",self.conv4_1,512,trainable=True)
        self.conv4_3 = self.conv("conv4_3",self.conv4_2,512,trainable=True)
        self.conv4_4 = self.conv("conv4_4",self.conv4_3,512,trainable=True)
        self.pool4 = self.maxpool("pool4",self.conv4_4,trainable=True)
        
        # Block 5
        self.conv5_1 = self.conv("conv5_1",self.pool4,512,trainable=True)
        self.conv5_2 = self.conv("conv5_2",self.conv5_1,512,trainable=True)
        self.conv5_3 = self.conv("conv5_3",self.conv5_2,512,trainable=True)
        self.conv5_4 = self.conv("conv5_4",self.conv5_3,512,trainable=True)
        self.pool5 = self.maxpool("pool5",self.conv5_4,trainable=True)
        
        # model modification for cifar-10
        self.fc6 = self.fc("fc6", self.pool5, 4096, relu_flag=True, trainable=True)
        if is_training == True:
            self.fc6 = tf.layers.dropout(self.fc6, 0.5)
        self.fc7 = self.fc("fc7", self.fc6, 4096, relu_flag=True, trainable=True)
        if is_training == True:
            self.fc7 = tf.layers.dropout(self.fc7, 0.5)
        self.fc8 = self.fc("fc8", self.fc7, classnum, relu_flag=False, trainable=True)
    
    def load_npz_weights(self, weight_file, sess):
        weights = np.load(weight_file)
        keys = sorted(weights.keys())
        for i, k in enumerate(keys):
            if i not in [36,37]:#最后一层不参加训练
                sess.run(self.parameters[i].assign(weights[k]))
        print("-----------Load weights done!---------------")
    def load_npy_weights(self, weight_file, sess):
        weights = np.load(weight_file, encoding = "bytes").item()
        #for layers in model
        for name in weights:
            if name not in self.skip:
                with tf.variable_scope(name, reuse = True):
                    for p in weights[name]:
                        if len(p.shape) == 1:
                            #bias 只有一维
                            sess.run(tf.get_variable('biases', trainable = True).assign(p))
                        else:
                            #weights
                            sess.run(tf.get_variable('weights', trainable = True).assign(p)) 

In [ ]:
### 用tfrecord训练 ###
#训练参数
BATCH_SIZE = 128
EPOCHS = 500             #迭代次数
EARLY_STOP_PATIENCE = 10 #控制early stopping的参数
NUM_CLASSS = 10

tf.reset_default_graph()
x_data = tf.placeholder(tf.float32, [None, 224, 224, 3])
y_data = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)
ckpt_path = './model/vgg/mode.ckpt'

vgg = VGG_19(x_data, 10, is_training)
predict = vgg.probs

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict, labels=y_data))
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):#批归一化层
    train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)
acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(predict, 1), tf.argmax(y_data, 1)), tf.float32))

#从TFrecord中导入数据队列
#img_batch,label_batch = TFRecord2TrainData2(train_tfrecords_path, NUM_CLASSS, BATCH_SIZE, EPOCHS)#每次读入1 epoch
img_train_batch, label_train_batch = TFRecord2TrainData2(train_tfrecords_path, NUM_CLASSS, BATCH_SIZE, EPOCHS)
img_val_batch, label_val_batch = TFRecord2TrainData2(val_tfrecords_path, NUM_CLASSS, BATCH_SIZE, EPOCHS)
#img_batch, label_batch = tf.train.shuffle_batch([img, label], batch_size=BATCH_SIZE, capacity=50000, min_after_dequeue=500, num_threads=1)

saver = tf.train.Saver()
with tf.Session() as sess:
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    sess.run(init_op)
    vgg.load_npy_weights(pre_model_path, sess)

    best_val_loss = 1000000.0
    current_epoch = 0

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    try:
        train_total_batch = int(train_num/BATCH_SIZE)
        val_total_bath = int(val_num/128)
        epoch = 0
        while not coord.should_stop() and epoch < EPOCHS:
            #开始一个epoch的训练
            train_loss_sum = 0
            for i in range(train_total_batch):
                x_train_batch, y_train_batch = sess.run([img_train_batch, label_train_batch])
                _, train_loss_epoch = sess.run([train_step, loss], \
                             feed_dict={x_data:x_train_batch, y_data:y_train_batch, is_training:True})
                train_loss_sum+=train_loss_epoch
            train_loss = train_loss_sum/train_total_batch 
            #每经过一个epoch，在验证集上进行验证
            val_acc_sum = 0
            val_loss_sum = 0
            for j in range(val_total_bath):
                x_val_batch, y_val_batch = sess.run([img_val_batch, label_val_batch])
                val_loss_epoch, val_acc_epoch = sess.run([loss, acc], feed_dict={x_data:x_val_batch, y_data:y_val_batch, is_training:False})
                val_acc_sum+=val_acc_epoch
                val_loss_sum+=val_loss_epoch
            val_loss = val_loss_sum/val_total_bath
            val_acc = val_acc_sum/val_total_bath
            print("epoch %d train loss is %f validataion loss is %f accuracy is %f" % (epoch, train_loss, val_loss, val_acc))
           
            if val_loss < best_val_loss:
                print('----  epoch %d current best_validation_loss is %f' % (epoch, val_loss))
                best_val_loss = val_loss
                current_epoch = epoch
                saver.save(sess, ckpt_path)
            elif (epoch - current_epoch) >= EARLY_STOP_PATIENCE:
                print('early stoping')
                break  
                
            epoch += 1
    except tf.errors.OutOfRangeError:
        print('done')
    finally:
        coord.request_stop()
        coord.join(threads)

epoch 0 train loss is 0.491314 validataion loss is 0.512945 accuracy is 0.829928
----  epoch 0 current best_validation_loss is 0.512945


In [7]:
### 计算混淆矩阵  ###
def list_flatten(a):
    b = []
    for each in a:
        if not isinstance(each, list):
            b.append(each)
        else:
            b.extend(list_flatten(each))
    return b  
def testModel(ckpt_path):
    tf.reset_default_graph()#mo
    x_data = tf.placeholder(tf.float32, [None, 224, 224, 3])
    y_data = tf.placeholder(tf.float32, [None, 10])
    is_training = tf.placeholder(tf.bool)
    vgg = VGG_19(x_data, 10, is_training)
    predict = vgg.probs
    
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(predict, 1), tf.argmax(y_data, 1)), tf.float32))
    
    img_batch,label_batch = TFRecord2TrainData2(val_tfrecords_path, 10, 128, 1, False)#每次读入1 epoch
    
    saver = tf.train.Saver() 
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        saver.restore(sess, ckpt_path)
        
        acc = []
        y_pred = []
        y_true = []
        
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)   
        try:
            while not coord.should_stop():
                x_batch, y_batch = sess.run([img_batch, label_batch])
                y_hat_batch = predict.eval({x_data: x_batch, is_training:False})
                y_pred_batch = np.argmax(y_hat_batch, axis=1)
                y_pred.append(y_pred_batch.tolist())
                y_true_batch = np.argmax(y_batch, axis=1)
                y_true.append(y_true_batch.tolist())
                
                acc_batch = accuracy.eval(feed_dict={x_data:x_batch, y_data:y_batch, is_training:False})
                acc.append(acc_batch.tolist())        
        
        except tf.errors.OutOfRangeError:
            print('done')
        finally:
            coord.request_stop()
            coord.join(threads)
            
        y_true_array = np.array(list_flatten(y_true))
        y_pred_array = np.array(list_flatten(y_pred))
        cm = confusion_matrix(y_true_array, y_pred_array)
        print(cm)
        print(acc)

In [8]:
ckpt_path = './model/vgg19/mode.ckpt'
testModel(ckpt_path)

INFO:tensorflow:Restoring parameters from ./model/vgg19/mode.ckpt
done
[[418   6  29  14   8   0   0   2  12   2]
 [  3 456   0   3   2   1   0   0   9  11]
 [ 12   1 433   4  11   3   8   5   1   0]
 [ 20   7  36 391   7  33   8  11   5   1]
 [ 13   0  77  16 332   3   7  22   4   1]
 [ 13   4  48  82  10 338   5  27   2   0]
 [  5   5  30  34   3   2 403   1   2   2]
 [  3   0  14   7  18   6   2 430   5   0]
 [ 66  17   3   5   8   0   2   2 417  12]
 [  3  27   1   1   0   0   0   0   4 475]]
[0.8203125, 0.8046875, 0.828125, 0.75, 0.828125, 0.828125, 0.890625, 0.7890625, 0.796875, 0.8046875, 0.8515625, 0.78125, 0.8203125, 0.8671875, 0.8359375, 0.8125, 0.796875, 0.8125, 0.859375, 0.8203125, 0.8828125, 0.8125, 0.734375, 0.8515625, 0.875, 0.828125, 0.7890625, 0.828125, 0.8671875, 0.8203125, 0.8515625, 0.84375, 0.7734375, 0.859375, 0.7734375, 0.8125, 0.7578125, 0.8203125, 0.796875]


In [9]:
### 测试集数据的读取  ###
#由于数据集较大，采用tensorflow自带的TFRecord进行读取
def testset2TFRecord(test_tfrecords_path, data_path='data'):
    files = natsorted(glob.glob(data_path + '/test/*'))
    total = len(files)
    if (os.path.exists(test_tfrecords_path)):
        print('data have already processed')
        return total
    writer = tf.python_io.TFRecordWriter(test_tfrecords_path)

    pbar = ProgressBar().start()
    for i, file_path in enumerate(files):
        pbar.update(int((i / (total - 1)) * 100))#进度条
        time.sleep(0.01)
        
        img_raw = tf.gfile.FastGFile(file_path, 'rb').read()
        example = tf.train.Example(features=tf.train.Features(feature={
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        }))
        writer.write(example.SerializeToString())
    writer.close()
    pbar.finish()
def TFRecord2TestData(test_tfrecords_path, _batch_size=128):
    filename_queue = tf.train.string_input_producer([test_tfrecords_path], num_epochs=1)
    #第二个参数num_epochs: 可选参数，是一个整数值，代表迭代的次数，
    #如果设置 num_epochs=None,生成器可以无限次遍历tensor列表，如果设置为 num_epochs=N，生成器只能遍历tensor列表N次。
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)   #返回文件名和文件
    features = tf.parse_single_example(serialized_example,
                                   features={
                                       'img_raw' : tf.FixedLenFeature([], tf.string)
                                   })
    img = tf.image.decode_png(features['img_raw'])  # 与方式一的不同点在于需要用decode_png/decode_jpeg解码
                                                    # output an RGB image. [height, width, channels]
    img = tf.reshape(img, [32, 32, 3])
    img = tf.image.resize_images(img, [224, 224])
    img_batch = tf.train.batch([img], batch_size=_batch_size, capacity=3*_batch_size)
    return img_batch

In [13]:
test_tfrecords_path = 'data/test.tfrecord'
test_num = testset2TFRecord(test_tfrecords_path)
print(test_num)

data have already processed
300000


In [20]:
### Submit ###
def submit(model_save_path, test_tfrecords_path, output_file):
    tf.reset_default_graph()
    x_test_data = tf.placeholder(tf.float32, [None, 224, 224, 3])
    is_training = tf.placeholder(tf.bool)
    vgg = VGG_19(x_test_data, 10, is_training)
    predict = vgg.probs
    
    batch_size = 128
    img_batch = TFRecord2TestData(test_tfrecords_path, batch_size)
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        saver.restore(sess, model_save_path)

        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess, coord)

        f = open(output_file, 'w')
        f.write('id,label\n')
        idx = 0
        try:
            pbar = ProgressBar().start()
            while not coord.should_stop():
                pbar.update(int((idx / (test_num - 1)) * 100))#进度条
                time.sleep(0.01)
                # just plot one batch size
                x_test_batch = sess.run(img_batch)
                y_hat = predict.eval({x_test_data: x_test_batch, is_training:False})
                y_pred_batch = np.argmax(y_hat, axis=1)
                for j in range(len(y_pred_batch)):
                    f.write("".join([str(idx+j), ',', int2lable[y_pred_batch[j]], '\n']))                
                idx+=len(y_pred_batch)
        except tf.errors.OutOfRangeError:
            print('done!')
            pbar.finish()
        finally:
            coord.request_stop()
            coord.join(threads)
        f.close()
    print('Finish!')

In [21]:
output_file = "submission.csv"
submit(ckpt_path, test_tfrecords_path, output_file)

INFO:tensorflow:Restoring parameters from ./model/vgg19/mode.ckpt


done!
Finish!
